In [1]:
import pandas as pd
df = pd.read_csv("/content/sample_data/USA_Housing.csv.crdownload")

df.info()
display(df.head())
display(df.describe())
display(df.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
dtypes: float64(6)
memory usage: 234.5 KB


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,68583.108984,5.977222,6.987792,3.981330,36163.516039,1.232073e+06
std,10657.991214,0.991456,1.005833,1.234137,9925.650114,3.531176e+05
min,17796.631190,2.644304,3.236194,2.000000,172.610686,1.593866e+04
25%,61480.562390,5.322283,6.299250,3.140000,29403.928700,9.975771e+05
50%,68804.286405,5.970429,7.002902,4.050000,36199.406690,1.232669e+06
75%,75783.338665,6.650808,7.665871,4.490000,42861.290770,1.471210e+06
max,107701.748400,9.519088,10.759588,6.500000,69621.713380,2.469066e+06


,0
Avg. Area Income,5000
Avg. Area House Age,5000
Avg. Area Number of Rooms,5000
Avg. Area Number of Bedrooms,255
Area Population,5000
Price,5000


In [2]:
X = df.drop(columns=["Price"]).values
y = df["Price"].values.reshape(-1, 1)  # convert it into 2D vector

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled=scaler.fit_transform(X) #for normalization (mean = 0 , sd = 1)

In [4]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split

In [5]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [6]:
import numpy as np
all_r2_scores = []  # To store the R² for every fold
best_r2 = -np.inf # to calculate max value we initialise with -infinity
best_beta = None

In [9]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score
import numpy as np

best_r2 = -np.inf
best_beta = None
all_test_r2_scores = []

for fold, (train_index, test_index) in enumerate(kf.split(X_scaled)):
    # Split into training and testing
    X_fold_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_fold_train, y_test = y[train_index], y[test_index]

    # Further split training into train and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_fold_train, y_fold_train, test_size=0.2, random_state=fold
    )

    # Add bias term
    X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_val_bias   = np.c_[np.ones((X_val.shape[0], 1)), X_val]
    X_test_bias  = np.c_[np.ones((X_test.shape[0], 1)), X_test]

    # Closed-form OLS solution
    beta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train

    # Validation predictions
    y_val_pred = X_val_bias @ beta
    val_r2 = r2_score(y_val, y_val_pred)
    print(f"Fold {fold}: Validation R² Score = {val_r2:.4f}")

    # Track best beta on validation
    if val_r2 > best_r2:
        best_r2 = val_r2
        best_beta = beta

    # Test predictions
    y_test_pred = X_test_bias @ beta
    test_r2 = r2_score(y_test, y_test_pred)
    all_test_r2_scores.append(test_r2)
    print(f"Fold {fold}: Test R² Score = {test_r2:.4f}")

# Summary
print(f"\nAverage Test R² Score: {np.mean(all_test_r2_scores):.4f}")
print(f"Standard Deviation of Test R² Scores: {np.std(all_test_r2_scores):.4f}")
print(f"Best Validation R² Score: {best_r2:.4f}")


Fold 0: Validation R² Score = 0.9122
Fold 0: Test R² Score = 0.9181
Fold 1: Validation R² Score = 0.9288
Fold 1: Test R² Score = 0.9145
Fold 2: Validation R² Score = 0.9198
Fold 2: Test R² Score = 0.9116
Fold 3: Validation R² Score = 0.9198
Fold 3: Test R² Score = 0.9192
Fold 4: Validation R² Score = 0.9122
Fold 4: Test R² Score = 0.9243

Average Test R² Score: 0.9175
Standard Deviation of Test R² Scores: 0.0043
Best Validation R² Score: 0.9288


In [11]:
# Final hold-out split (20% test data)
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Add bias column for intercept
X_test_final_bias = np.c_[np.ones((X_test_final.shape[0], 1)), X_test_final]

# Predictions using best beta
y_pred_final = X_test_final_bias @ best_beta

# R² score
final_r2 = r2_score(y_test_final, y_pred_final)
print(f"\nFinal R² Score on 20% test data: {final_r2:.4f}")



Final R² Score on 20% test data: 0.9184


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Load dataset
df = pd.read_csv("/content/sample_data/USA_Housing.csv.crdownload")

# Separate features and target
X = df.drop('Price', axis=1)
Y = df['Price']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data: 56% train, 14% val, 30% test
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X_scaled, Y, test_size=0.44, random_state=42
)
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp, test_size=(30/44), random_state=42
)

# Convert targets to NumPy arrays
y_train, y_val, y_test = Y_train.values, Y_val.values, Y_test.values

# Add bias column
X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_val_b   = np.c_[np.ones((X_val.shape[0], 1)), X_val]
X_test_b  = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# Gradient Descent
def gradient_descent(X, y, learning_rate, n_iterations):
    m = X.shape[0]
    beta = np.random.randn(X.shape[1])  # random init
    for iteration in range(n_iterations):
        gradients = (2/m) * X.T @ (X @ beta - y)
        beta = beta - learning_rate * gradients
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
best_beta = None
best_r2_val = -np.inf

print("--- Training with Gradient Descent for different learning rates ---")
for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}")
    beta_gd = gradient_descent(X_train_b, y_train, lr, 1000)

    y_val_pred = X_val_b @ beta_gd
    r2_val = r2_score(y_val, y_val_pred)
    print(f"R2 score on validation set: {r2_val:.4f}")

    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_beta = beta_gd

print("\n--- Final Evaluation ---")
print("Best Regression Coefficients (beta):", best_beta)

y_test_pred = X_test_b @ best_beta
r2_test = r2_score(y_test, y_test_pred)
print(f"R2 score on test set with best coefficients: {r2_test:.4f}")


--- Training with Gradient Descent for different learning rates ---

Training with learning rate: 0.001
R2 score on validation set: 0.6451

Training with learning rate: 0.01
R2 score on validation set: 0.9200

Training with learning rate: 0.1
R2 score on validation set: 0.9200

Training with learning rate: 1
R2 score on validation set: -inf

--- Final Evaluation ---
Best Regression Coefficients (beta): [1232180.27200919  230645.88389435  165328.94019375  120045.00851908
    2945.02108903  151375.22971285]
R2 score on test set with best coefficients: 0.9134


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1275: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)


In [14]:
import pandas as pd
import numpy as np

# Column names
columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
           "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
           "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
           "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(url, names=columns, na_values='?')

In [15]:
# Drop rows where price is NaN
df.dropna(subset=['price'], inplace=True)

# Convert price to numeric
df['price'] = pd.to_numeric(df['price'])

# Impute missing values with mean (for numeric columns)
for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col].fillna(df[col].mean(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)

/tmp/ipython-input-1700601401.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/tmp/ipython-input-1700601401.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [16]:
word_to_num = {
    'two': 2, 'three': 3, 'four': 4, 'five': 5,
    'six': 6, 'eight': 8, 'twelve': 12
}

df['num_doors'] = df['num_doors'].map(word_to_num)
df['num_cylinders'] = df['num_cylinders'].map(word_to_num)

In [17]:
df = pd.get_dummies(df, columns=['body_style', 'drive_wheels'], drop_first=True)


In [18]:
from sklearn.preprocessing import LabelEncoder

for col in ['make', 'aspiration', 'engine_location', 'fuel_type']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [19]:
df['fuel_system'] = df['fuel_system'].apply(lambda x: 1 if 'pfi' in x else 0)


In [20]:
df['engine_type'] = df['engine_type'].apply(lambda x: 1 if 'ohc' in x else 0)


In [21]:
from sklearn.preprocessing import StandardScaler

X = df.drop('price', axis=1)
Y = df['price']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train, Y_train)
Y_pred = lr.predict(X_test)

print("R² Score (Original Features):", r2_score(Y_test, Y_pred))

R² Score (Original Features): 0.8732775682086301


In [23]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # retain 95% variance
X_pca = pca.fit_transform(X_scaled)

X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(X_pca, Y, test_size=0.3, random_state=42)

lr_pca = LinearRegression()
lr_pca.fit(X_train_pca, Y_train_pca)
Y_pred_pca = lr_pca.predict(X_test_pca)

print("R² Score (PCA Features):", r2_score(Y_test_pca, Y_pred_pca))

R² Score (PCA Features): 0.8611839960452383
